In [3]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Student(Person):
    def __init__(self, name, age, id, semester, section):
        super().__init__(name, age)
        self.id = id
        self.semester = semester
        self.section = section

    def study(self):
        print("studying")


In [4]:
students = [ Student("Abrar", 20, 12142352, "3-1", "B"),
            Student("XYZ", 22, 1242352, "3-2", "A")]

In [6]:
for i in students:
    print(f"name:{i.name} ")
    i.study()

name:Abrar 
studying
name:XYZ 
studying


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

class HousingDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, y

data = fetch_california_housing()
X = data['data']
y = data['target']

# Perform data preprocessing
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = HousingDataset(X_train, y_train)
test_dataset = HousingDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = NeuralNetwork(input_size=X.shape[1])

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# Display a sample of the dataset
print("Sample from the dataset:")
sample_idx = 0
sample_input, sample_target = train_dataset[sample_idx]
print("Input:", sample_input)
print("Target:", sample_target)
print()

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Compute average training loss
    train_loss /= len(train_loader)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    test_correct = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)

            test_loss += loss.item()

    # Compute average test loss
    test_loss /= len(test_loader)

    # Compute test accuracy
    test_accuracy = 100.0 - test_loss

    # Print training and testing statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Test Accuracy: {test_accuracy:.2f}%")

Sample from the dataset:
Input: tensor([-0.3217,  0.3465, -0.1663, -0.1905,  0.7723,  0.0598, -1.3680,  1.2676])
Target: tensor(1.0300)



100%|██████████| 258/258 [00:00<00:00, 421.35it/s]


Epoch [1/10], Train Loss: 1.7177, Test Accuracy: 99.29%


100%|██████████| 258/258 [00:00<00:00, 460.77it/s]


Epoch [2/10], Train Loss: 0.6347, Test Accuracy: 99.46%


...

Epoch [8/10], Train Loss: 0.3800, Test Accuracy: 99.61%


100%|██████████| 258/258 [00:00<00:00, 454.16it/s]


Epoch [9/10], Train Loss: 0.3733, Test Accuracy: 99.62%


100%|██████████| 258/258 [00:00<00:00, 448.56it/s]


Epoch [10/10], Train Loss: 0.3676, Test Accuracy: 99.62%
